# Praxisübung: Regularisierung und Hyperparametersuche

*In den folgenden Aufgaben werden Sie mehrere Modelle trainieren, wobei jeder Trainingslauf --- je nach Anzahl der Features und Anzahl der Datenpunkte --- eine gewisse Zeit benötigt. Um die Modellentwicklung zu beschleunigen ist es hilfreich, zunächst eine zufällig ausgewählte Teilmenge des Datensatzes zu betrachten (z.B. 10000 Punkte), die Sie sich mittels `DataFrame.sample(10000)` erzeugen.*

## ✏ Aufgabe 1
Wenden Sie Ridge Regression und Lasso für das Wohnungsbeispiel aus der letzten Woche an. Weiter unten finden Sie eine überarbeitete, übersichtlichere Variante des Immoscout Notebooks. Sie können aber natürlich auch ihre eigene Version aus der vorherigen Übung benutzen.

1. Skalieren Sie die Features mit dem `StandardScaler` aus `scikit-learn`. Das Skalieren sollte nach dem Aufteilen in Trainings- und Testset passieren. Schauen Sie sich dazu die Dokumentation unter https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html an.

2. Trainieren Sie Modelle mit $\ell_2$- und mit $\ell_1$-Regularisierung (Ridge Regression und Lasso). Schauen Sie in die Dokumentation https://scikit-learn.org/stable/modules/linear_model.html, um die geeigneten Methoden dafür zu finden.

3. Beide Methoden haben einen Parameter `alpha`. Wie ist der Wert jeweils zu interpretieren (siehe Dokumentation)?
    
4. Vergleichen Sie Modelle aus der vorherigen Teilaufgabe ($R^2$ bzw.\ Fehler auf den Trainings- bzw. Testdaten) für verschiedene Werte von `alpha`. Wie können Sie einen möglichst "guten" Wert für den Parameter finden?
    
5. Zählen Sie bei der Modellauswertung für die verschiedenen $\ell_2$- bzw. $\ell_1$-regularisierten Modelle zusätzlich die Anzahl der Koeffizienten, die ungleich Null sind. Was beobachten Sie?


## ✏ Aufgabe 2
Führen Sie eine Hyperparametersuche mit Kreuzvalidierung durch, indem Sie den Algorithmus aus der Vorlesung für das Immoscout Beispiel implementieren.
Sie können sich herantasten, indem Sie zuerst die einfache Hyperparametersuche (ohne Kreuzvalidierung) implementieren und dann Schritt 2 des Algorithmus ersetzen.
Benutzen Sie die KFold Klasse aus scikit-learn um sich die Datensätze für die Kreuzvalidierung zu erzeugen: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.htm.


## ✏ Aufgabe 3
Führen Sie eine nested-Cross Validation durch, um eine bessere Schätzung des Testfehlers zu erhalten. D.h. anstatt *ein* Trainings- und Testset zu erzeugen (vor der Hyperparametersuche), erzeugen Sie $k$ Testsets. Für jedes dieser $k$ Trainings-/Testset Paare führen Sie nun die Hyperparametersuche mit Kreuzvalidierung wie in der vorherigen Aufgabe durch. Als Resultat erhalten Sie eine Schätzung des Testfehlers durch Mittelung der $k$ Testfehler und $k$ Modelle.


## ✏ Aufgabe 4 🤯
Zeigen Sie: Für ein gegebenes $\lambda > 0$ wird die Verlustfunktion für die Ridge Regression
\begin{align*}
    L(w)&=\sum_{i=1}^N\left(y_i-w^Tx_i\right)^2 + \sum_{i=1}^p w_i^2 \\&= \left\|y-Xw\right\|^2 + \lambda\left\|w\right\|^2
\end{align*}
minimiert von
\begin{align*}
    w=(X^TX+\lambda I)^{-1}X^Ty,
\end{align*}
wobei $I\in\R^{p\times p}$ die Einheitsmatrix bezeichnet.

Anleitung: Orientieren Sie sich dabei am Skript, wo das entsprechende Resultat für die lineare Regression (ohne Regularisierungsterm) hergeleitet wird.


In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn import linear_model

from IPython.display import display

pd.options.display.max_columns = 50

In [4]:
df = pd.read_csv("Daten/immo_data.csv")
desc = pd.read_csv("Daten/immo_data_column_description.csv")

In [10]:
def drop_columns(df):
    """ Entfernen (vermeintlich) unwichtiger Spalten """
    return df.drop(
        [
            "scoutId",
            "houseNumber",
            "geo_bln",
            "geo_krs",
            "geo_plz",
            "date",
            "street",
            "streetPlain",
            "description",
            "facilities",
            "regio3",
            "firingTypes",
            "telekomHybridUploadSpeed",
            "totalRent",
            "baseRentRange",
        ],
        axis=1,
    )


def remove_outliers(df, lower_limit=0.005, upper_limit=0.995):
    """ Entfernen der (unteren und oberen) Ausreißer """
    dfc = df.copy()
    columns_with_outliers = [
        "serviceCharge",
        "yearConstructed",
        "noParkSpaces",
        "baseRent",
        "livingSpace",
        "noRooms",
        "floor",
        "numberOfFloors",
        "heatingCosts",
        "lastRefurbish",
    ]
    
    # Für jede Spalte behalten wir: Daten die < (99.5%-Quantil) sind und > (0.5%-Quantil) sind ODER die NaN sind (damit befassen wir uns spaeter noch) 
    upper_limits = df[columns_with_outliers].quantile(upper_limit)
    lower_limits = df[columns_with_outliers].quantile(lower_limit)
    
    for colname in columns_with_outliers:
        col = dfc[colname]
        dfc = dfc[
            ((col <= upper_limits[colname]) & (col >= lower_limits[colname]))
            | col.isna()
        ]
    return dfc


def remove_rows_with_NaN_target(df):
    """ Entfernen der Datensätze ohne Label"""
    return df[df["baseRent"].isna() == False]


def impute_NaNs(df):
    """ Ersetzen von NaNs durch Mittelwert bzw. Modus """
    dfc = df.copy()
    categorical_columns = dfc.select_dtypes(exclude=np.number).columns
    imp_freq = SimpleImputer(missing_values=np.nan, strategy="most_frequent")
    dfc.loc[:, categorical_columns] = imp_freq.fit_transform(dfc[categorical_columns])

    numeric_columns = dfc.select_dtypes(include=np.number).columns
    imp_mean = SimpleImputer(missing_values=np.nan, strategy="mean")
    dfc.loc[:, numeric_columns] = imp_mean.fit_transform(dfc[numeric_columns])
    return dfc


def print_evaluation(pipeline_or_model, X_train, X_test, y_train, y_test, y_train_pred, y_test_pred, feature_names):
    """ Ausgabe von R2-Wert, MSE und MAE für Trainings- und Testset """
    r2_train = r2_score(y_train, y_train_pred)
    mse_train = mean_squared_error(y_train, y_train_pred)
    mae_train = mean_absolute_error(y_train, y_train_pred)

    r2_test = r2_score(y_test, y_test_pred)
    mse_test = mean_squared_error(y_test, y_test_pred)
    mae_test = mean_absolute_error(y_test, y_test_pred)
    
    print(
        f"{pipeline_or_model} Evaluation:\n"
        f"{'':6} {'R²':>10} | {'MSE':>14} | {'MAE':>10} | {'rows':>8} | {'columns':>8}\n"
        f"{'Train':6} {r2_train:10.5f} | {mse_train:14.2f} | {mae_train:10.2f} | {X_train.shape[0]:8} | {X_train.shape[1]:8}\n"
        f"{'Test':6} {r2_test:10.5f} | {mse_test:14.2f} | {mae_test:10.2f} | {X_test.shape[0]:8} | {X_test.shape[1]:8}\n"
    )
    
    # Ausgabe der ersten 10 Koeffizienten, absteigend nach Absolutbetrag sortiert
    coefficients_lr = pd.DataFrame({"Feature Name": feature_names, "Coefficient": pipeline_or_model.coef_})
    display(coefficients_lr.sort_values("Coefficient", key=abs, ascending=False).head(10))

In [11]:
# Datenvorverarbeitung
df_reduced = drop_columns(df.sample(10000))
df_reduced = remove_outliers(df_reduced)
df_reduced = remove_rows_with_NaN_target(df_reduced)
df_reduced = impute_NaNs(df_reduced)
df_reduced = pd.get_dummies(df_reduced)
y = df_reduced.pop("baseRent")

# Training-Test-Split
X_train, X_test, y_train, y_test = train_test_split(df_reduced, y, test_size=0.2, random_state=0)

# Training
model_lr = linear_model.LinearRegression()
model_lr.fit(X_train, y_train)
y_train_pred = model_lr.predict(X_train)
y_test_pred = model_lr.predict(X_test)

# Evaluation
print_evaluation(model_lr, X_train, X_test, y_train, y_test, y_train_pred, y_test_pred, feature_names=df_reduced.columns)

LinearRegression() Evaluation:
               R² |            MSE |        MAE |     rows |  columns
Train     0.84047 |       26718.59 |     106.33 |   207773 |      512
Test      0.84079 |       27084.82 |     107.07 |    51944 |      512



,Feature Name,Coefficient
66,interiorQual_luxury,3.084705e+11
69,interiorQual_sophisticated,3.084705e+11
67,interiorQual_normal,3.084705e+11
68,interiorQual_simple,3.084705e+11
28,regio1_Bremen,-1.611739e+11
132,regio2_Bremen,1.538304e+11
133,regio2_Bremerhaven,1.538304e+11
510,energyEfficiencyClass_H,1.514252e+11
504,energyEfficiencyClass_B,1.514252e+11
509,energyEfficiencyClass_G,1.514252e+11
